In [5]:
! unzip drive/MyDrive/SI.zip

Archive:  drive/MyDrive/SI.zip
   creating: dataset/
   creating: dataset/flooded/
  inflating: dataset/flooded/0.jpg   
 extracting: dataset/flooded/1.jpg   
  inflating: dataset/flooded/10.jpg  
  inflating: dataset/flooded/100.jpg  
  inflating: dataset/flooded/101.jpg  
  inflating: dataset/flooded/102.jpg  
  inflating: dataset/flooded/103.jpg  
  inflating: dataset/flooded/104.jpg  
  inflating: dataset/flooded/105.jpg  
 extracting: dataset/flooded/106.jpg  
  inflating: dataset/flooded/107.jpg  
  inflating: dataset/flooded/108.jpg  
  inflating: dataset/flooded/109.jpg  
  inflating: dataset/flooded/11.jpg  
  inflating: dataset/flooded/110.jpg  
  inflating: dataset/flooded/111.jpg  
  inflating: dataset/flooded/112.jpg  
  inflating: dataset/flooded/113.jpg  
 extracting: dataset/flooded/114.jpg  
  inflating: dataset/flooded/115.jpg  
  inflating: dataset/flooded/116.jpg  
  inflating: dataset/flooded/117.jpg  
  inflating: dataset/flooded/118.jpg  
  inflating: dataset/flo

In [35]:
import tensorflow as tf

# Define the paths to the dataset folders
flooded_images_dir = "dataset/flooded"
non_flooded_images_dir = "dataset/non-flooded"

# Define the desired image size
image_size = (256, 256)

# Create a list of image file paths for flooded images
flooded_image_paths = tf.data.Dataset.list_files(flooded_images_dir + '/*')

# Create a list of image file paths for non-flooded images
non_flooded_image_paths = tf.data.Dataset.list_files(non_flooded_images_dir + '/*')

flooded_images_dataset = flooded_image_paths.map(lambda x: tf.io.read_file(x))
flooded_images_dataset = flooded_images_dataset.map(lambda x: tf.image.decode_jpeg(x, channels=3))
flooded_images_dataset = flooded_images_dataset.map(lambda x: tf.image.resize(x, image_size))
flooded_images_dataset = flooded_images_dataset.map(lambda x: (x, 1))  # Set label as 1 for flooded images

# Read and resize the non-flooded images
non_flooded_images_dataset = non_flooded_image_paths.map(lambda x: tf.io.read_file(x))
non_flooded_images_dataset = non_flooded_images_dataset.map(lambda x: tf.image.decode_jpeg(x, channels=3))
non_flooded_images_dataset = non_flooded_images_dataset.map(lambda x: tf.image.resize(x, image_size))
non_flooded_images_dataset = non_flooded_images_dataset.map(lambda x: (x, 0))  # Set label as 0 for non-flooded images
# Combine the flooded and non-flooded datasets
dataset = tf.data.Dataset.concatenate(flooded_images_dataset, non_flooded_images_dataset)


# Shuffle and batch the dataset
dataset = dataset.shuffle(buffer_size=1000)

split_ratio = 0.8
# Determine the number of samples for train and validation based on the split ratio
total_samples = len(list(dataset))
train_samples = int(total_samples * split_ratio)
validation_samples = total_samples - train_samples

# Split the dataset into train and validation subsets
train_dataset = dataset.take(train_samples)
validation_dataset = dataset.skip(train_samples)

# Batch the datasets
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
validation_dataset = validation_dataset.batch(batch_size)



In [36]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [42]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the ResNet block
def resnet_block(inputs, filters, strides=1, activation='relu'):
    x = layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Shortcut connection
    shortcut = inputs
    if strides > 1:
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation(activation)(x)
    return x

# Define the ResNet model
def resnet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    x = resnet_block(x, filters=64)
    x = resnet_block(x, filters=64)
    x = resnet_block(x, filters=64)

    x = resnet_block(x, filters=128, strides=2)
    x = resnet_block(x, filters=128)
    x = resnet_block(x, filters=128)

    x = resnet_block(x, filters=256, strides=2)
    x = resnet_block(x, filters=256)
    x = resnet_block(x, filters=256)

    x = resnet_block(x, filters=512, strides=2)
    x = resnet_block(x, filters=512)
    x = resnet_block(x, filters=512)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

# Define the paths to the dataset folders
flooded_images_dir = "/path/to/flooded_images_folder"
non_flooded_images_dir = "/path/to/non_flooded_images_folder"

# Define the desired image size
image_size = (256, 256)

# Define the number of classes
num_classes = 1

input_shape = (256, 256, 3)

# Create an instance of the ResNet model
model = resnet(input_shape, num_classes)


In [43]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_56 (Conv2D)             (None, 128, 128, 64  9472        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_56 (BatchN  (None, 128, 128, 64  256        ['conv2d_56[0][0]']              
 ormalization)                  )                                                           

In [44]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [45]:
epochs = 10
steps_per_epoch = train_samples // batch_size
validation_steps = validation_samples // batch_size

hist = model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)


Epoch 1/10
24/24 [==============================] - 84s 2s/step - loss: 1.1733 - accuracy: 0.6445 - val_loss: 11200948.0000 - val_accuracy: 0.5135
Epoch 2/10
24/24 [==============================] - 63s 1s/step - loss: 0.4706 - accuracy: 0.7707 - val_loss: 100151.9688 - val_accuracy: 0.4757
Epoch 3/10
24/24 [==============================] - 75s 2s/step - loss: 0.3728 - accuracy: 0.8385 - val_loss: 750.4586 - val_accuracy: 0.4919
Epoch 4/10
24/24 [==============================] - 65s 2s/step - loss: 0.4448 - accuracy: 0.8100 - val_loss: 413.2361 - val_accuracy: 0.5514
Epoch 5/10
24/24 [==============================] - 61s 1s/step - loss: 0.3500 - accuracy: 0.8630 - val_loss: 28.0262 - val_accuracy: 0.4811
Epoch 6/10
24/24 [==============================] - 61s 1s/step - loss: 0.3293 - accuracy: 0.8684 - val_loss: 53.5816 - val_accuracy: 0.5081
Epoch 7/10
24/24 [==============================] - 65s 2s/step - loss: 0.3243 - accuracy: 0.8657 - val_loss: 4.9625 - val_accuracy: 0.4973
Ep

In [46]:
hist2 = model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)


Epoch 1/10
24/24 [==============================] - 62s 1s/step - loss: 0.2377 - accuracy: 0.9104 - val_loss: 3.1304 - val_accuracy: 0.5838
Epoch 2/10
24/24 [==============================] - 63s 1s/step - loss: 0.2460 - accuracy: 0.9091 - val_loss: 1.4665 - val_accuracy: 0.5622
Epoch 3/10
24/24 [==============================] - 75s 2s/step - loss: 0.2364 - accuracy: 0.9037 - val_loss: 0.7788 - val_accuracy: 0.6649
Epoch 4/10
24/24 [==============================] - 62s 1s/step - loss: 0.2493 - accuracy: 0.9077 - val_loss: 0.7940 - val_accuracy: 0.7568
Epoch 5/10
24/24 [==============================] - 75s 2s/step - loss: 0.1943 - accuracy: 0.9308 - val_loss: 1.6545 - val_accuracy: 0.6703
Epoch 6/10
24/24 [==============================] - 64s 2s/step - loss: 0.2821 - accuracy: 0.9186 - val_loss: 1.8659 - val_accuracy: 0.6649
Epoch 7/10
24/24 [==============================] - 63s 1s/step - loss: 0.2494 - accuracy: 0.9091 - val_loss: 1.9278 - val_accuracy: 0.5676
Epoch 8/10
24/24 [==